# 3/20
- やったこと
    - 	age	index_group_no	top12のテーブルを作った

In [1]:
import pandas as pd
import numpy as np

In [2]:
# データの読み込み
DATA_DIR = "../data/original/"
articles_df=pd.read_pickle(f'{DATA_DIR}articles.pkl')
customers_df=pd.read_pickle(f'{DATA_DIR}customers.pkl')
transactions_train_df=pd.read_pickle(f'{DATA_DIR}transactions_train.pkl')

In [3]:
articles_df = articles_df.head(10000)
customers_df = customers_df.head(10000)
transactions_train_df = transactions_train_df.head(10000)

In [4]:
# articles と customersをtransactionsを使って結合する
def create_features(articles_df,customers_df,transactions_train_df):
    tt_temp = transactions_train_df[["customer_id","article_id"]]
    a_df = articles_df[["article_id","index_group_no"]]
    c_df = customers_df[["customer_id","age"]]
    at_merged_df =  pd.merge(tt_temp,a_df)
    features_df = pd.merge(c_df,at_merged_df)
    features_df = features_df.dropna(how="all")
    return features_df

In [5]:
# １０までとってくる
def convert_age_into_10_delimited(df):
    df["age"] = df["age"] / 10
    df["age"] = df["age"].astype(np.int64) 
    df["age"] = df["age"] * 10
    return df

In [6]:
features_df = create_features(articles_df,customers_df,transactions_train_df)

In [7]:
features_df = convert_age_into_10_delimited(features_df)

In [8]:
# top 12のarticles_id
articles_most_buy_30 = list(features_df['article_id'].value_counts().head(30).index)

In [9]:
df = features_df

# ageとindex?group

In [10]:
count = 0

df = df.groupby(["age","index_group_no"])["article_id"].agg(lambda x:list(x)).reset_index()
df

,age,index_group_no,article_id
0,20,1,"[501323011, 507909001, 524825010, 186262006, 2..."
1,20,2,"[501820043, 501820043, 377277001, 377277001, 4..."
2,20,3,[397068015]
3,20,4,[427114015]
4,20,26,[500780006]
5,30,1,"[464277014, 464277014, 522561002, 445419009, 5..."
6,30,2,"[505221004, 505221001, 518827004, 487827009, 5..."
7,40,1,"[508184020, 179208001, 244267032]"
8,40,2,[512251004]
9,40,3,"[534795002, 427159007]"


In [11]:
# 上位12のarticle idをとってきておく
TOP_N = 12
def calc_top(ids):
    ids= sorted(ids)
    if len(ids) < TOP_N:
        return [*ids, *articles_most_buy_30][0:TOP_N]
    else:
        return ids[0:TOP_N]
df[[f"top{i}" for i in range(1,TOP_N+1)]] = df.apply(lambda x: calc_top(x["article_id"]), axis = 1,result_type='expand')
df = df.drop(["article_id"],axis=1)

In [12]:
df

,age,index_group_no,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12
0,20,1,186262006,214844001,297078001,457892005,496111020,501323011,502186011,507909001,524825010,536162002,537688014,501820043
1,20,2,212042043,377277001,377277001,399136061,399136061,448509017,501820043,501820043,501820043,377277001,464277014,399136061
2,20,3,397068015,501820043,377277001,464277014,399136061,211143037,503729006,505221004,487053021,535877018,400319005,445419009
3,20,4,427114015,501820043,377277001,464277014,399136061,211143037,503729006,505221004,487053021,535877018,400319005,445419009
4,20,26,500780006,501820043,377277001,464277014,399136061,211143037,503729006,505221004,487053021,535877018,400319005,445419009
5,30,1,445419009,464277014,464277014,506110001,522561002,532578008,501820043,377277001,464277014,399136061,211143037,503729006
6,30,2,400319005,487827009,505221001,505221004,518827004,527687006,535877018,501820043,377277001,464277014,399136061,211143037
7,40,1,179208001,244267032,508184020,501820043,377277001,464277014,399136061,211143037,503729006,505221004,487053021,535877018
8,40,2,512251004,501820043,377277001,464277014,399136061,211143037,503729006,505221004,487053021,535877018,400319005,445419009
9,40,3,427159007,534795002,501820043,377277001,464277014,399136061,211143037,503729006,505221004,487053021,535877018,400319005
